In [43]:
import pymongo

DCE_DB_NAME = "DCEdb"
DCE_COMMODITY_PRICE_COLLECTION_NAME = "CommodityPriceDb"
DCE_FUTURE_PRICE_COLLECTION_NAME = "DCEFuturePriceDb"
DCE_FUTURE_CODE_COLLECTION_NAME = 'DCEFutureCodeDb'
COSMOS_CONNECTION_STRING = 'mongodb://lei:WMNi1Hh4vyOVG4V3OLcyaoZwOOl7YVHUUiKFeuGKbiS9rytxi2qd2VpnekCYj8IVrN6AFVHvLBUxACDb7vuFnA==@lei.mongo.cosmos.azure.com:10255/?ssl=true&replicaSet=globaldb&retrywrites=false&maxIdleTimeMS=600000&appName=@lei@'


In [44]:
client = pymongo.MongoClient(COSMOS_CONNECTION_STRING)
DCEdb = client[DCE_DB_NAME]

In [45]:
pipeline = [
    {"$match":{"date":{"$eq":'2023-09-11'}}}, 
    { "$group": { "_id": { "future_code": "$future_code", "clock": "$clock", "date": "$date" }, 
                 "_idsNeedsToBeDeleted": { "$push": "$$ROOT._id" } 
                 } }, 
    { "$project": { "_id": 0, 
                   "_idsNeedsToBeDeleted": { "$slice": ["$_idsNeedsToBeDeleted", 1, { "$size": "$_idsNeedsToBeDeleted" }] } } },
    { "$unwind": "$_idsNeedsToBeDeleted"  },
    { "$group": { "_id": "", "_idsNeedsToBeDeleted": { "$push": "$_idsNeedsToBeDeleted" } } }, 
    { "$project": { "_id": 0 } },
    {"$limit":1000}
]

In [46]:
a = list(DCEdb.DCEFuturePriceDb.aggregate(pipeline))

In [47]:
k = a[0]['_idsNeedsToBeDeleted']

In [48]:
k

[ObjectId('64ffa9108f0d92e817f42933'),
 ObjectId('64ffa9c48f0d92e817f42ba9'),
 ObjectId('64ffaa008f0d92e817f42c7b'),
 ObjectId('64ffaab48f0d92e817f42ef1'),
 ObjectId('64ffaba48f0d92e817f43239'),
 ObjectId('64ffabe08f0d92e817f4330b'),
 ObjectId('64ffacd08f0d92e817f43653'),
 ObjectId('64ffad848f0d92e817f438c9'),
 ObjectId('64ffadfc8f0d92e817f43a6d'),
 ObjectId('64ffae388f0d92e817f43b3f'),
 ObjectId('64ffaeb08f0d92e817f43ce3'),
 ObjectId('64ffaf648f0d92e817f43f59'),
 ObjectId('64ffafa08f0d92e817f4402b'),
 ObjectId('64ffafdc8f0d92e817f440fd'),
 ObjectId('64ffb0188f0d92e817f441cf'),
 ObjectId('64ffb1448f0d92e817f445e9'),
 ObjectId('64ffb1808f0d92e817f446bb'),
 ObjectId('64ffb1f88f0d92e817f4485f'),
 ObjectId('64ffb2348f0d92e817f44931'),
 ObjectId('64ffb2708f0d92e817f44a03'),
 ObjectId('64ffb2e88f0d92e817f44ba7'),
 ObjectId('64ffb3608f0d92e817f44d4b'),
 ObjectId('64ffb39c8f0d92e817f44e1d'),
 ObjectId('64ffb3d88f0d92e817f44eef'),
 ObjectId('64ffb4508f0d92e817f45093'),
 ObjectId('64ffb5408f0d92

In [49]:
DCE_future_price_collection = DCEdb[DCE_FUTURE_PRICE_COLLECTION_NAME]
DCE_future_price_collection.delete_many( { "_id" : {"$in" : k[0:1000]} } )